In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from pathlib import Path
import pandas as pd
from bramm_data_analysis import loading
import matplotlib.pyplot as plt
import gstlearn as gl
import gstlearn.plot as gp
import gstlearn.document as gdoc
import gstools as gs

verbose = True
graphics = True
gl.OptCst.define(gl.ECst.NTCOL, 6)
gdoc.setNoScroll()

In [ ]:
columns = {}

columns["metadata"] = [
    "site_code",
    "site_insee_code",
    "department_code",
    "latitude",
    "longitude",
    "x_lambert",
    "y_lambert",
    "altitude",
    "date",
    "sample_code",
    "sample_send_europe",
]
columns["sample"] = [
    "strand_size",
    "visible_dust_particles",
    "visible_pollen_particles",
    "mineral_type",
]
columns["site"] = [
    "samples_nb",
    "fern_samples_nb",
    "tree_samples_nb",
    "herb_samples_nb",
    "litter_samples_nb",
    "humus_samples_nb",
    "soi_samples_nb",
    "sand_samples_nb",
    "hard_strain_coniferous_samples_nb",
    "hard_strain_hardwood_samples_nb",
    "hard_strain_unknown_samples_nb",
    "decomposed_strain_coniferous_samples_nb",
    "decomposed_strain_hardwood_samples_nb",
    "decomposed_strain_unknown_samples_nb",
    "bark_coniferous_nb",
    "bark_hardwood_samples_nb",
    "bark_unknown_samples_nb",
    "hard_coniferous_samples_nb",
    "hard_hardwood_samples_nb",
    "hard_unknown_samples_nb",
    "decomposed_coniferous_samples_nb",
    "decomposed_hardwood_samples_nb",
    "decomposed_unknown_samples_nb",
]
columns["studies"] = [
    "sample_outside_complementary_study",
    "cs_3_species_comparison",
    "cs_2_species_comparison",
    "cs_repeated_sampling",
    "cs_repeated_analysis",
]
columns["incertitudes"] = [
    "aluminium_incertitude",
    "arsenic_incertitude",
    "calcium_incertitude",
    "cadmium_incertitude",
    "cobalt_incertitude",
    "chromium_incertitude",
    "copper_incertitude",
    "iron_incertitude",
    "mercury_incertitude",
    "nitrogen_incertitude",
    "sodium_incertitude",
    "nickel_incertitude",
    "lead_incertitude",
    "sulfur_incertitude",
    "zinc_incertitude",
]
columns["categorical"] = [
    "weather",
    "tree_layer",
    "tree_layer_complement",
    "tree_cover",
    "species",
]

In [ ]:
df_sites = loading.load_sites(Path("../data/Mines_2024.xlsx"))
df_samples = loading.load_samples(Path("../data/Mines_2024.xlsx"))
df_values = loading.load_values(Path("../data/Mines_2024.xlsx"))

In [ ]:
df = pd.merge(
    pd.merge(
        df_sites,
        df_samples,
        on="site_code",
    ),
    df_values,
    on="sample_code",
)
# Correct Data
df.loc[
    df["tree_layer_complement"].isna(), "tree_layer_complement"
] = "unspecified"

df = df.drop(columns=columns["incertitudes"])

# Correct data
for col in [
    "sodium",
    "platinium",
    "rhodium",
    "antimony",
    "strontium",
    "vanadium",
    "zinc",
]:
    df[col] = df[col].astype(str)
    df[col] = df[col].apply(lambda x: x.replace(",", "."))
    df[col] = df[col].apply(lambda x: x.replace("< ", ""))
    df[col].astype("float64")

# Remove Columns with nans
print("The following columns contains NaN and will be removed :")
print(df.columns[df.isna().any(axis=0)])
df = df.drop(columns=df.columns[df.isna().any(axis=0)])

In [ ]:
df.columns

In [ ]:
x1 = "longitude"
x2 = "latitude"
z1 = "iron"
z2 = "aluminium"
raw_df = df.filter([x1, x2, z1, z2]).astype("float")
final_df = raw_df[(raw_df[z2] <= 8000) & (raw_df[z1] <= 4000)]

In [ ]:
mydb = gl.Db_fromPanda(final_df)
mydb.setLocators([x1, x2], gl.ELoc.X)
mydb.setLocator(z1, gl.ELoc.Z)

In [ ]:
mydb

In [ ]:
gl.dbStatisticsPrint(
    db=mydb,
    names=[z1, z2],
    opers=gl.EStatOption.fromKeys(["MEAN", "MINI", "MAXI"]),
    flagIso=False,
    title="Number of observations",
)

In [ ]:
ax = gp.correlation(mydb, namex=z1, namey=z2, asPoint=True, regrLine=True)
ax.decoration(title=f"Correlation between {z1} and {z2}")

In [ ]:
varioparam = gl.VarioParam.createMultiple(
    ndir=2, npas=30, dpas=0.2, toldis=0.6
)
vario_raw2dir = gl.Vario(varioparam)
err = vario_raw2dir.compute(mydb)

In [ ]:
gp.setDefault(dims=[6, 6])
gp.varmod(vario_raw2dir)
plt.show()

In [ ]:
fitmod_raw = gl.Model()
err = fitmod_raw.fit(
    vario_raw2dir,
    types=[gl.ECov.NUGGET, gl.ECov.EXPONENTIAL, gl.ECov.CUBIC, gl.ECov.LINEAR],
)
fitmod_raw.setDriftIRF(0, 0)

In [ ]:
ax = gp.varmod(vario_raw2dir, fitmod_raw)
ax.decoration(
    title=f"Experimental and fitted variogram models - Raw {z2} observations",
    xlabel="Distance (km)",
    ylabel="Variogram",
)